In [23]:
!pip install accelerate

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [24]:
import torch
from datetime import datetime
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [25]:
checkpoint = "bigcode/starcoder2-7b"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# for fp16 use `torch_dtype=torch.float16` instead
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto", torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [26]:
def generateOutputWithModelPipeline(model, tokenizer, prompt, temperature=0.7) : 
    pipe = pipeline(
        task="text-generation", 
        model=model, 
        tokenizer=tokenizer
    )
    
    start = datetime.now()
    
    output = pipe(
        prompt,
        do_sample=True,
        max_new_tokens=256, 
        temperature=temperature, 
        top_k=50, 
        top_p=0.95,
        num_return_sequences=1
    )
    
    stop = datetime.now()
    
    totalTimeToPrompt = stop - start
    print(f"Execution Time : {totalTimeToPrompt}")
    
    return output

In [27]:
prompt = """
Question: Write a script in Python that implements a circular queue and creates one with user input.
Answer:
"""

In [ ]:
generateOutputWithModelPipeline(model, tokenizer, prompt)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [ ]:
%%time
inputs = tokenizer.encode(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(inputs)
print(tokenizer.decode(outputs[0]))